In [4]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("../include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    origen_anuncio = 'WebScraping'
    --and marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# Mostramos los primeros registros:
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60479478,RENAULT,clio,11990.0,Diésel,2022.0,123000.0,100.0,5.0,5.0,...,Berlina,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...
1,60374025,CITROEN,c3,16500.0,Gasolina,2022.0,18150.0,83.0,5.0,5.0,...,Berlina,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...
2,60576644,VOLKSWAGEN,California Ocean 2.0 TDI BMT DSG,67900.0,Diesel,2023.0,12000.0,150.0,4.0,4.0,...,Monovolumen,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...
3,60550981,OPEL,Corsa 1.4 Selective 90 CV,5900.0,Gasolina,2014.0,176864.0,90.0,5.0,5.0,...,Berlina,1398.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-05-17,202505.0,2025-05-18 07:23:57,https://www.coches.net/opel-corsa-1.4-selectiv...
4,60266007,CITROEN,Berlingo Talla M BlueHDi 100 LIVE,13990.0,Diesel,2019.0,155700.0,100.0,4.0,5.0,...,Monovolumen,1499.0,Blanco,Pontevedra,None,WebScraping,2025-05-17,202505.0,2025-05-18 07:23:31,https://www.coches.net/citroen-berlingo-talla-...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568 entries, 0 to 1567
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        1568 non-null   int64  
 1   marca                1568 non-null   object 
 2   modelo               1568 non-null   object 
 3   precio               1568 non-null   float64
 4   combustible          1520 non-null   object 
 5   anyo_vehiculo        1564 non-null   float64
 6   kilometraje          1564 non-null   float64
 7   potencia             1547 non-null   float64
 8   num_puertas          1552 non-null   float64
 9   num_plazas           1548 non-null   float64
 10  tipo_cambio          1564 non-null   object 
 11  tipo_vehiculo        1564 non-null   object 
 12  cilindrada_motor     1532 non-null   float64
 13  color                1532 non-null   object 
 14  provincia            1564 non-null   object 
 15  etiqueta_eco         1120 non-null   o

Aplicamos las transformaciones que faltan:

### 1. Correción modelos:

1. Conecta con la misma base de datos de vehículos
2. Carga los modelos estandarizados usando la misma consulta SQL
3. Implementa la función de comparación y estandarización
4. Proporciona una función estandarizar_modelos_df() que puedes aplicar a cualquier DataFrame que tenga columnas 'marca' y 'modelo'

In [19]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect("../include/db_vehiculos.db")
cursor = conn.cursor()

# Cargar modelos estandarizados desde la base de datos
def cargar_modelos_estandarizados():
    cursor.execute("""
        SELECT marca, modelo 
        FROM TX_VEHICULOS_SEG_MANO 
        WHERE origen_anuncio = 'dataset kaggle' 
        GROUP BY marca, modelo
    """)
    
    modelos_por_marca = {}
    for marca, modelo in cursor.fetchall():
        if marca not in modelos_por_marca:
            modelos_por_marca[marca] = []
        modelos_por_marca[marca].append(modelo.lower().replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u"))
    
    print(f"📚 Cargados {sum(len(modelos) for modelos in modelos_por_marca.values())} modelos estandarizados de {len(modelos_por_marca)} marcas")
    return modelos_por_marca

# Función para encontrar el modelo estandarizado
def encontrar_modelo_estandarizado(marca, modelo_original, modelos_por_marca):
    marca_upper = marca.upper()
    modelo_tokens = set(modelo_original.lower().replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").split())
    
    if marca_upper not in modelos_por_marca:
        return modelo_original
    
    modelos = modelos_por_marca[marca_upper]
    mejor_modelo = modelo_original
    max_coincidencias = 0
    
    # Aquí devuelve el modelo en el que coincidan el mayor número de tokens del string
    for modelo_estandar in modelos:
        modelo_estandar_tokens = set(modelo_estandar.split())
        coincidencias = len(modelo_tokens.intersection(modelo_estandar_tokens))
        
        if coincidencias > max_coincidencias:
            max_coincidencias = coincidencias
            mejor_modelo = modelo_estandar
    
    return mejor_modelo



# Cargar el diccionario de modelos estandarizados
modelos_estandarizados = cargar_modelos_estandarizados()

# Función para aplicar al DataFrame
def estandarizar_modelos_df(df):
    # Crear copia del DataFrame para no modificar el original
    df_estandarizado = df.copy()
    
    # Crear columna para mostrar los modelos originales y estandarizados
    df_estandarizado['modelo_original'] = df_estandarizado['modelo']
    
    # Contador de cambios
    cambios = 0
    total = len(df_estandarizado)
    
    # Aplicar la estandarización a cada fila
    for idx, row in df_estandarizado.iterrows():
        if idx % 1000 == 0:
            print(f"Procesando fila {idx}/{total}...")
            
        marca = row['marca']
        modelo_original = row['modelo']
        
        if pd.isna(marca) or pd.isna(modelo_original):
            continue
            
        modelo_estandarizado = encontrar_modelo_estandarizado(
            str(marca), 
            str(modelo_original), 
            modelos_estandarizados
        )
        
        if modelo_estandarizado != modelo_original:
            df_estandarizado.at[idx, 'modelo'] = modelo_estandarizado
            cambios += 1
    
    print(f"✅ Estandarización completada. Se modificaron {cambios} modelos de {total} registros ({cambios/total*100:.2f}%)")
    
    # Mostrar algunos ejemplos de cambios realizados
    muestra_cambios = df_estandarizado[df_estandarizado['modelo'] != df_estandarizado['modelo_original']].head(10)
    if len(muestra_cambios) > 0:
        print("\nEjemplos de cambios realizados:")
        for _, row in muestra_cambios.iterrows():
            print(f"- {row['marca']} | Original: '{row['modelo_original']}' -> Estandarizado: '{row['modelo']}'")
    
    return df_estandarizado

📚 Cargados 384 modelos estandarizados de 10 marcas


In [21]:
modelos_estandarizados["SEAT"]

['altea',
 'alhambra',
 'altea freetrack',
 'altea xl',
 'arona',
 'arosa',
 'ateca',
 'cordoba',
 'exeo',
 'ibiza',
 'inca',
 'leon',
 'malaga',
 'marbella',
 'mii',
 'tarraco',
 'terra',
 'toledo']

In [22]:
estandarizar_modelos_df(df)

Procesando fila 0/1568...
Procesando fila 1000/1568...
✅ Estandarización completada. Se modificaron 1550 modelos de 1568 registros (98.85%)

Ejemplos de cambios realizados:
- VOLKSWAGEN | Original: 'California Ocean 2.0 TDI BMT DSG' -> Estandarizado: 'california'
- OPEL | Original: 'Corsa 1.4 Selective 90 CV' -> Estandarizado: 'corsa'
- CITROEN | Original: 'Berlingo Talla M BlueHDi 100 LIVE' -> Estandarizado: 'berlingo'
- MERCEDES-BENZ | Original: 'Clase SLK SLK 200 K' -> Estandarizado: 'clase slk'
- FORD | Original: 'Kuga 1.5 EcoBoost 4x2 Titanium' -> Estandarizado: 'kuga'
- VOLKSWAGEN | Original: 'Polo Sport 1.4 TDI 66kW90CV BMT' -> Estandarizado: 'polo'
- VOLKSWAGEN | Original: 'California Beach Camper TDI BMT' -> Estandarizado: 'california'
- AUDI | Original: 'A3 30 TDI S tronic Sportback' -> Estandarizado: 'a3'
- AUDI | Original: 'A3 Sportback 30 TFSI S tronic' -> Estandarizado: 'a3'
- CITROEN | Original: 'Berlingo Talla M BlueHDi 100 SHINE' -> Estandarizado: 'berlingo'


,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url,modelo_original
0,60479478,RENAULT,clio,11990.0,Diésel,2022.0,123000.0,100.0,5.0,5.0,...,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...,clio
1,60374025,CITROEN,c3,16500.0,Gasolina,2022.0,18150.0,83.0,5.0,5.0,...,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...,c3
2,60576644,VOLKSWAGEN,california,67900.0,Diesel,2023.0,12000.0,150.0,4.0,4.0,...,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...,California Ocean 2.0 TDI BMT DSG
3,60550981,OPEL,corsa,5900.0,Gasolina,2014.0,176864.0,90.0,5.0,5.0,...,1398.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-05-17,202505.0,2025-05-18 07:23:57,https://www.coches.net/opel-corsa-1.4-selectiv...,Corsa 1.4 Selective 90 CV
4,60266007,CITROEN,berlingo,13990.0,Diesel,2019.0,155700.0,100.0,4.0,5.0,...,1499.0,Blanco,Pontevedra,None,WebScraping,2025-05-17,202505.0,2025-05-18 07:23:31,https://www.coches.net/citroen-berlingo-talla-...,Berlingo Talla M BlueHDi 100 LIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,59973168,BMW,serie 1,9990.0,Diesel,2013.0,142133.0,116.0,5.0,5.0,...,1995.0,Azul,Cantabria,B (amarilla),WebScraping,2025-02-27,202502.0,2025-05-11 17:46:05,https://www.coches.net/bmw-serie-1-116d-5p-die...,Serie 1 116d
1564,60439306,BMW,serie 3,84900.0,Gasolina,2023.0,26000.0,510.0,4.0,5.0,...,2993.0,Gris / Plata,Pontevedra,None,WebScraping,2025-03-05,202503.0,2025-05-11 17:37:18,https://www.coches.net/bmw-serie-3-m3-competit...,Serie 3 M3 Competition
1565,59030836,BMW,serie 3,18890.0,Diesel,2019.0,114331.0,150.0,5.0,5.0,...,1995.0,Negro,Madrid,C (verde),WebScraping,2025-04-04,202504.0,2025-05-11 17:37:10,https://www.coches.net/bmw-serie-3-318d-tourin...,Serie 3 318d Touring
1566,60330276,BMW,serie 3,34990.0,Híbrido Enchufable,2020.0,66011.0,292.0,5.0,5.0,...,1998.0,Azul,Barcelona,None,WebScraping,2025-04-12,202504.0,2025-05-11 17:37:02,https://www.coches.net/bmw-serie-3-330e-xdrive...,Serie 3 330e xDrive Touring


In [25]:
df_estandarizado = estandarizar_modelos_df(df)

df_estandarizado.drop(columns=["modelo_original"], inplace=True)

df_estandarizado.head()

Procesando fila 0/1568...
Procesando fila 1000/1568...
✅ Estandarización completada. Se modificaron 1550 modelos de 1568 registros (98.85%)

Ejemplos de cambios realizados:
- VOLKSWAGEN | Original: 'California Ocean 2.0 TDI BMT DSG' -> Estandarizado: 'california'
- OPEL | Original: 'Corsa 1.4 Selective 90 CV' -> Estandarizado: 'corsa'
- CITROEN | Original: 'Berlingo Talla M BlueHDi 100 LIVE' -> Estandarizado: 'berlingo'
- MERCEDES-BENZ | Original: 'Clase SLK SLK 200 K' -> Estandarizado: 'clase slk'
- FORD | Original: 'Kuga 1.5 EcoBoost 4x2 Titanium' -> Estandarizado: 'kuga'
- VOLKSWAGEN | Original: 'Polo Sport 1.4 TDI 66kW90CV BMT' -> Estandarizado: 'polo'
- VOLKSWAGEN | Original: 'California Beach Camper TDI BMT' -> Estandarizado: 'california'
- AUDI | Original: 'A3 30 TDI S tronic Sportback' -> Estandarizado: 'a3'
- AUDI | Original: 'A3 Sportback 30 TFSI S tronic' -> Estandarizado: 'a3'
- CITROEN | Original: 'Berlingo Talla M BlueHDi 100 SHINE' -> Estandarizado: 'berlingo'


,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60479478,RENAULT,clio,11990.0,Diésel,2022.0,123000.0,100.0,5.0,5.0,...,Berlina,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...
1,60374025,CITROEN,c3,16500.0,Gasolina,2022.0,18150.0,83.0,5.0,5.0,...,Berlina,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...
2,60576644,VOLKSWAGEN,california,67900.0,Diesel,2023.0,12000.0,150.0,4.0,4.0,...,Monovolumen,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...
3,60550981,OPEL,corsa,5900.0,Gasolina,2014.0,176864.0,90.0,5.0,5.0,...,Berlina,1398.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-05-17,202505.0,2025-05-18 07:23:57,https://www.coches.net/opel-corsa-1.4-selectiv...
4,60266007,CITROEN,berlingo,13990.0,Diesel,2019.0,155700.0,100.0,4.0,5.0,...,Monovolumen,1499.0,Blanco,Pontevedra,None,WebScraping,2025-05-17,202505.0,2025-05-18 07:23:31,https://www.coches.net/citroen-berlingo-talla-...


In [26]:
df_estandarizado["modelo"].value_counts()

modelo
corsa             140
c3                128
clio              103
ibiza             101
megane             83
                 ... 
transporter         1
crossland x         1
s-max               1
tourneo custom      1
x2                  1
Name: count, Length: 108, dtype: int64

In [38]:
marcas_permitidas = ['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']

df_estandarizado = df_estandarizado[df_estandarizado["marca"].isin(marcas_permitidas)]


In [39]:
df_estandarizado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1564 entries, 0 to 1567
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        1564 non-null   int64  
 1   marca                1564 non-null   object 
 2   modelo               1564 non-null   object 
 3   precio               1564 non-null   float64
 4   combustible          1520 non-null   object 
 5   anyo_vehiculo        1564 non-null   float64
 6   kilometraje          1564 non-null   float64
 7   potencia             1547 non-null   float64
 8   num_puertas          1552 non-null   float64
 9   num_plazas           1548 non-null   float64
 10  tipo_cambio          1564 non-null   object 
 11  tipo_vehiculo        1564 non-null   object 
 12  cilindrada_motor     1532 non-null   float64
 13  color                1532 non-null   object 
 14  provincia            1564 non-null   object 
 15  etiqueta_eco         1120 non-null   object

Ya tenemos los modelos bien, vamos a recargar la BBDD con ellos bien para seguir con el .ipynb del modelo:

In [35]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    origen_anuncio = 'WebScraping'
    --origen_anuncio = 'dataset kaggle'
    --and marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
    --pk_anuncio_id in (1, 2, 3, 4)
order by
    fecha_carga desc
;
"""

df_db = pd.read_sql_query(query, con)

df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60479478,RENAULT,clio,11990.0,Diésel,2022,123000,100.0,5.0,5.0,...,Berlina,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...
1,60374025,CITROEN,c3,16500.0,Gasolina,2022,18150,83.0,5.0,5.0,...,Berlina,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...
2,60576644,VOLKSWAGEN,California Ocean 2.0 TDI BMT DSG,67900.0,Diesel,2023,12000,150.0,4.0,4.0,...,Monovolumen,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...
3,60550981,OPEL,Corsa 1.4 Selective 90 CV,5900.0,Gasolina,2014,176864,90.0,5.0,5.0,...,Berlina,1398.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-05-17,202505.0,2025-05-18 07:23:57,https://www.coches.net/opel-corsa-1.4-selectiv...
4,60266007,CITROEN,Berlingo Talla M BlueHDi 100 LIVE,13990.0,Diesel,2019,155700,100.0,4.0,5.0,...,Monovolumen,1499.0,Blanco,Pontevedra,None,WebScraping,2025-05-17,202505.0,2025-05-18 07:23:31,https://www.coches.net/citroen-berlingo-talla-...


In [36]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        1564 non-null   int64  
 1   marca                1564 non-null   object 
 2   modelo               1564 non-null   object 
 3   precio               1564 non-null   float64
 4   combustible          1520 non-null   object 
 5   anyo_vehiculo        1564 non-null   int64  
 6   kilometraje          1564 non-null   int64  
 7   potencia             1547 non-null   float64
 8   num_puertas          1552 non-null   float64
 9   num_plazas           1548 non-null   float64
 10  tipo_cambio          1564 non-null   object 
 11  tipo_vehiculo        1564 non-null   object 
 12  cilindrada_motor     1532 non-null   float64
 13  color                1532 non-null   object 
 14  provincia            1564 non-null   object 
 15  etiqueta_eco         1120 non-null   o

In [40]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    origen_anuncio = 'WebScraping'
    --origen_anuncio = 'dataset kaggle'
    --marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
    --pk_anuncio_id in (1, 2, 3, 4)
order by
    fecha_carga desc
;
"""

df_db_new = pd.read_sql_query(query, con)

df_db_new.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60479478,RENAULT,clio,11990.0,Diésel,2022,123000,100.0,5.0,5.0,...,Berlina,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...
1,60374025,CITROEN,c3,16500.0,Gasolina,2022,18150,83.0,5.0,5.0,...,Berlina,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...
2,60576644,VOLKSWAGEN,California Ocean 2.0 TDI BMT DSG,67900.0,Diesel,2023,12000,150.0,4.0,4.0,...,Monovolumen,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...
3,60550981,OPEL,Corsa 1.4 Selective 90 CV,5900.0,Gasolina,2014,176864,90.0,5.0,5.0,...,Berlina,1398.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-05-17,202505.0,2025-05-18 07:23:57,https://www.coches.net/opel-corsa-1.4-selectiv...
4,60266007,CITROEN,Berlingo Talla M BlueHDi 100 LIVE,13990.0,Diesel,2019,155700,100.0,4.0,5.0,...,Monovolumen,1499.0,Blanco,Pontevedra,None,WebScraping,2025-05-17,202505.0,2025-05-18 07:23:31,https://www.coches.net/citroen-berlingo-talla-...


In [41]:
df_db_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        1564 non-null   int64  
 1   marca                1564 non-null   object 
 2   modelo               1564 non-null   object 
 3   precio               1564 non-null   float64
 4   combustible          1520 non-null   object 
 5   anyo_vehiculo        1564 non-null   int64  
 6   kilometraje          1564 non-null   int64  
 7   potencia             1547 non-null   float64
 8   num_puertas          1552 non-null   float64
 9   num_plazas           1548 non-null   float64
 10  tipo_cambio          1564 non-null   object 
 11  tipo_vehiculo        1564 non-null   object 
 12  cilindrada_motor     1532 non-null   float64
 13  color                1532 non-null   object 
 14  provincia            1564 non-null   object 
 15  etiqueta_eco         1120 non-null   o

In [ ]:
# Eliminamos los regs con el formato de "modelo" mal:

# import sqlite3

# con = sqlite3.connect("../include/db_vehiculos.db")
# cursor = con.cursor()

# delete_query = """
# DELETE FROM TX_VEHICULOS_SEG_MANO
# WHERE origen_anuncio = 'WebScraping';
# """

# cursor.execute(delete_query)
# con.commit()  # Muy importante para que los cambios se guarden
# con.close()

In [43]:
# Insertamos los que tienen el formato de "modelo" bien:

import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

try:
    df_estandarizado.to_sql('TX_VEHICULOS_SEG_MANO', con, if_exists='append', index=False)
    print("Datos insertados con éxito")
except Exception as e:
    print("Error al insertar datos:", e)

con.close()

Datos insertados con éxito


In [51]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
--where
    --origen_anuncio = 'WebScraping'
    --origen_anuncio = 'dataset kaggle'
    --marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
    --pk_anuncio_id in (1, 2, 3, 4)
order by
    fecha_carga desc
;
"""

df_db_new = pd.read_sql_query(query, con)

df_db_new.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,59710104,PEUGEOT,308,9000.0,Gasolina,2016.0,127300,110.0,5.0,5.0,...,Berlina,1199.0,blanco,Asturias,None,WebScraping,2025-01-25,202501.0,2025-05-18 11:18:51,https://www.coches.net/peugeot-308-5p-allure-1...
1,60479478,RENAULT,clio,11990.0,Diésel,2022.0,123000,100.0,5.0,5.0,...,Berlina,1461.0,blanco,Málaga,None,WebScraping,2025-05-18,202505.0,2025-05-18 10:06:26,https://www.coches.net/renault-clio-equilibre-...
2,60374025,CITROEN,c3,16500.0,Gasolina,2022.0,18150,83.0,5.0,5.0,...,Berlina,1199.0,blanco,Madrid,None,WebScraping,2025-04-19,202504.0,2025-05-18 10:05:00,https://www.coches.net/citroen-c3-puretech-60k...
3,60576644,VOLKSWAGEN,california,67900.0,Diesel,2023.0,12000,150.0,4.0,4.0,...,Monovolumen,1968.0,Blanco,Las Palmas,None,WebScraping,2025-05-18,202505.0,2025-05-18 07:24:21,https://www.coches.net/volkswagen-california-o...
4,60550981,OPEL,corsa,5900.0,Gasolina,2014.0,176864,90.0,5.0,5.0,...,Berlina,1398.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-05-17,202505.0,2025-05-18 07:23:57,https://www.coches.net/opel-corsa-1.4-selectiv...


In [52]:
df_db_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36606 entries, 0 to 36605
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        36606 non-null  int64  
 1   marca                36606 non-null  object 
 2   modelo               36606 non-null  object 
 3   precio               36606 non-null  float64
 4   combustible          36544 non-null  object 
 5   anyo_vehiculo        36605 non-null  float64
 6   kilometraje          36606 non-null  int64  
 7   potencia             29813 non-null  float64
 8   num_puertas          36594 non-null  float64
 9   num_plazas           1549 non-null   float64
 10  tipo_cambio          36526 non-null  object 
 11  tipo_vehiculo        1565 non-null   object 
 12  cilindrada_motor     1533 non-null   float64
 13  color                36127 non-null  object 
 14  provincia            36602 non-null  object 
 15  etiqueta_eco         1120 non-null  

In [60]:
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')
df_db_new[df_db_new["marca"] == "SEAT"]["modelo"].value_counts()

modelo
ibiza              925
leon               875
arona              234
ateca              144
toledo             101
alhambra            99
altea xl            89
altea               83
cordoba             77
exeo                54
tarraco             48
mii                 20
marbella            14
arosa               13
inca                10
altea freetrack      4
malaga               2
terra                2
Name: count, dtype: int64